## Argument Parsing CLI

This will help us parse the input from Users


In [2]:
import argparse


def get_input_args():
    # Creates parse
    parser = argparse.ArgumentParser()

    # Creates 3 command line arguments args.dir for path to images files,
    # args.arch which CNN model to use for classification, args.labels path to
    # text file with names of dogs.
    parser.add_argument('--dir',
                        type=str,
                        default='pet_images/',
                        help='path to folder of images')

    #          --arch - the CNN model architecture
    #          --dogfile - text file of names of dog breeds
    parser.add_argument('--arch', type=str, default='vgg')
    parser.add_argument('--dogfile', type=str, default='dognames.txt')

    # collection that you created with this function
    return parser.parse_args()

## Directory Scanner

### Helps us form the result dictionary


In [3]:
# */AIPND-revision/intropyproject-classify-pet-images/get_pet_labels.py
#
# PROGRAMMER: Abhijith Ganesh
# DATE CREATED: 28 November
# REVISED DATE:

from os import listdir

# TODO 2: Define get_pet_labels function below please be certain to replace None
#       in the return statement with results_dict dictionary that you create
#       with this function
#


def get_pet_labels(image_dir):
    """
    Creates a dictionary of pet labels (results_dict) based upon the filenames 
    of the image files. These pet image labels are used to check the accuracy 
    of the labels that are returned by the classifier function, since the 
    filenames of the images contain the true identity of the pet in the image.
    Be sure to format the pet labels so that they are in all lower case letters
    and with leading and trailing whitespace characters stripped from them.
    (ex. filename = 'Boston_terrier_02259.jpg' Pet label = 'boston terrier')
    Parameters:
     image_dir - The (full) path to the folder of images that are to be
                 classified by the classifier function (string)
    Returns:
      results_dict - Dictionary with 'key' as image filename and 'value' as a 
      List. The list contains for following item:
         index 0 = pet image label (string)
    """

    _dir = listdir(image_dir)
    _results = {}
    for i in _dir:
        _temp = i.split(".jpg")
        _str = _temp[0]
        _str = _str.lower()
        _results[_str] = "pet_images/" + i
    # Replace None with the results_dict dictionary that you created with this
    # function
    return _results

## Classifier with CNN

This is a Model Trainer, It uses PyTorch CNN


In [4]:
import ast
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.models as models
from torch import __version__

resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

models = {'resnet': resnet18, 'alexnet': alexnet, 'vgg': vgg16}

# obtain ImageNet labels
with open('imagenet1000_clsid_to_human.txt') as imagenet_classes_file:
    imagenet_classes_dict = ast.literal_eval(imagenet_classes_file.read())


def classifier(img_path, model_name):
    # load the image
    img_pil = Image.open(img_path)

    # define transforms
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # preprocess the image
    img_tensor = preprocess(img_pil)

    # resize the tensor (add dimension for batch)
    img_tensor.unsqueeze_(0)

    # wrap input in variable, wrap input in variable - no longer needed for
    # v 0.4 & higher code changed 04/26/2018 by Jennifer S. to handle PyTorch upgrade
    pytorch_ver = __version__.split('.')

    # pytorch versions 0.4 & hihger - Variable depreciated so that it returns
    # a tensor. So to address tensor as output (not wrapper) and to mimic the
    # affect of setting volatile = True (because we are using pretrained models
    # for inference) we can set requires_gradient to False. Here we just set
    # requires_grad_ to False on our tensor
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        img_tensor.requires_grad_(False)

    # pytorch versions less than 0.4 - uses Variable because not-depreciated
    else:
        # apply model to input
        # wrap input in variable
        data = Variable(img_tensor, volatile=True)

    # apply model to input
    model = models[model_name]

    # puts model in evaluation mode
    # instead of (default)training mode
    model = model.eval()

    # apply data to model - adjusted based upon version to account for
    # operating on a Tensor for version 0.4 & higher.
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        output = model(img_tensor)

    # pytorch versions less than 0.4
    else:
        # apply data to model
        output = model(data)

    # return index corresponding to predicted class
    pred_idx = output.data.numpy().argmax()

    return imagenet_classes_dict[pred_idx]


c:\Users\Abhijith G\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Abhijith G\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Abhijith G\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexN

## Classify Images with Model


In [83]:
# PURPOSE: Create a function classify_images that uses the classifier function
#          to create the classifier labels and then compares the classifier
#          labels to the pet image labels. This function inputs:
#            -The Image Folder as image_dir within classify_images and function
#             and as in_arg.dir for function call within main.
#            -The results dictionary as results_dict within classify_images
#             function and results for the functin call within main.
#            -The CNN model architecture as model wihtin classify_images function
#             and in_arg.arch for the function call within main.
#           This function uses the extend function to add items to the list
#           that's the 'value' of the results dictionary. You will be adding the
#           classifier label as the item at index 1 of the list and the comparison
#           of the pet and classifier labels as the item at index 2 of the list.
#
##

# from classifier import classifier

# TODO 3: Define classify_images function below, specifically replace the None


def classify_images(images_dir, results_dict, model):
    """

    Creates classifier labels with classifier function, compares pet labels to 
    the classifier labels, and adds the classifier label and the comparison of 
    the labels to the results dictionary using the extend function. Be sure to
    format the classifier labels so that they will match your pet image labels.
    The format will include putting the classifier labels in all lower case 
    letters and strip the leading and trailing whitespace characters from them.
    For example, the Classifier function returns = 'Maltese dog, Maltese terrier, Maltese' 
    so the classifier label = 'maltese dog, maltese terrier, maltese'.
    Recall that dog names from the classifier function can be a string of dog 
    names separated by commas when a particular breed of dog has multiple dog 
    names associated with that breed. For example, you will find pet images of
    a 'dalmatian'(pet label) and it will match to the classifier label 
    'dalmatian, coach dog, carriage dog' if the classifier function correctly 
    classified the pet images of dalmatians.

     PLEASE NOTE: This function uses the classifier() function defined in 
     classifier.py within this function. The proper use of this function is
     in test_classifier.py Please refer to this program prior to using the 
     classifier() function to classify images within this function 
     Parameters: 
      images_dir - The (full) path to the folder of images that are to be
                   classified by the classifier function (string)
      results_dict - Results Dictionary with 'key' as image filename and 'value'
                    as a List. Where the list will contain the following items: 
                  index 0 = pet image label (string)
                --- where index 1 & index 2 are added by this function ---
                  NEW - index 1 = classifier label (string)
                  NEW - index 2 = 1/0 (int)  where 1 = match between pet image
                    and classifer labels and 0 = no match between labels
      model - Indicates which CNN model architecture will be used by the 
              classifier function to classify the pet images,
              values must be either: resnet alexnet vgg (string)
     Returns:
           None - results_dict is mutable data type so no return needed.         
    """

    data = get_pet_labels(images_dir)

    for ky, val in data.items():
        _result = classifier(val, model)
        _split = _result.split(" ")
        _keys = ky.split("_")
        _file = images_dir + "\\" + ky

        if (_keys[0] in _split):
            results_dict[_file] = [
                _keys[0], _split[_split.index(_keys[0])].lower(), 1
            ]
        else:
            results_dict[_file] = [_keys[0], _result, 0]


results_dict = {}
classify_images(r"S:\udacity-aws-ai-ml\jupyter\pet_images", results_dict,
                'resnet')


### Testing Functions


In [237]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# */AIPND/intropylab-classifying-images/print_functions_for_lab_checks.py
#
# PROGRAMMER: Jennifer S.
# DATE CREATED: 05/14/2018
# REVISED DATE:             <=(Date Revised - if any)
# PURPOSE:  This set of functions can be used to check your code after programming
#           each function. The top section of each part of the lab contains
#           the section labeled 'Checking your code'. When directed within this
#           section of the lab one can use these functions to more easily check
#           your code. See the docstrings below each function for details on how
#           to use the function within your code.
#
##


# Functions below defined to help with "Checking your code", specifically
# running these functions with the appropriate input arguments within the
# main() funtion will print out what's needed for "Checking your code"
#
def check_command_line_arguments(in_arg):
    """
    For Lab: Classifying Images - 7. Command Line Arguments
    Prints each of the command line arguments passed in as parameter in_arg, 
    assumes you defined all three command line arguments as outlined in 
    '7. Command Line Arguments'
    Parameters:
     in_arg -data structure that stores the command line arguments object
    Returns:
     Nothing - just prints to console  
    """
    if in_arg is None:
        print(
            "* Doesn't Check the Command Line Arguments because 'get_input_args' hasn't been defined."
        )
    else:
        # prints command line agrs
        print("Command Line Arguments:\n     dir =", in_arg.dir,
              "\n    arch =", in_arg.arch, "\n dogfile =", in_arg.dogfile)


def check_creating_pet_image_labels(results_dict):
    """    For Lab: Classifying Images - 9/10. Creating Pet Image Labels
    Prints first 10 key-value pairs and makes sure there are 40 key-value 
    pairs in your results_dict dictionary. Assumes you defined the results_dict
    dictionary as was outlined in 
    '9/10. Creating Pet Image Labels'
    Parameters:
      results_dict - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
    Returns:
     Nothing - just prints to console  
    """
    if results_dict is None:
        print(
            "* Doesn't Check the Results Dictionary because 'get_pet_labels' hasn't been defined."
        )
    else:
        # Code to print 10 key-value pairs (or fewer if less than 10 images)
        # & makes sure there are 40 pairs, one for each file in pet_images/
        stop_point = len(results_dict)
        if stop_point > 10:
            stop_point = 10
        print("\nPet Image Label Dictionary has", len(results_dict),
              "key-value pairs.\nBelow are", stop_point, "of them:")

        # counter - to count how many labels have been printed
        n = 0

        # for loop to iterate through the dictionary
        for key in results_dict:

            # prints only first 10 labels
            if n < stop_point:
                print("{:2d} key: {:>30}  label: {:>26}".format(
                    n + 1, key, results_dict[key][0]))

                # Increments counter
                n += 1

            # If past first 10 (or fewer) labels the breaks out of loop
            else:
                break


def check_classifying_images(results_dict):
    """    For Lab: Classifying Images - 11/12. Classifying Images
    Prints Pet Image Label and Classifier Label for ALL Matches followed by ALL 
    NOT matches. Next prints out the total number of images followed by how 
    many were matches and how many were not-matches to check all 40 images are
    processed. Assumes you defined the results_dict dictionary as was 
    outlined in '11/12. Classifying Images'
    Parameters:
      results_dict - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)   where 1 = match between pet image and 
                    classifer labels and 0 = no match between labels
    Returns:
     Nothing - just prints to console  

    """
    if results_dict is None:
        print(
            "* Doesn't Check the Results Dictionary because 'classify_images' hasn't been defined."
        )
    elif len(results_dict[next(iter(results_dict))]) < 2:
        print(
            "* Doesn't Check the Results Dictionary because 'classify_images' hasn't been defined."
        )
    else:
        # Code for checking classify_images -
        # Checks matches and not matches are classified correctly
        # Checks that all 40 images are classified as a Match or Not-a Match

        # Sets counters for matches & NOT-matches
        n_match = 0
        n_notmatch = 0

        # Prints all Matches first
        print("\n     MATCH:")
        for key in results_dict:

            # Prints only if a Match Index 2 == 1
            if results_dict[key][2] == 1:

                # Increments Match counter
                n_match += 1
                print("\n{:>30}: \nReal: {:>26}   Classifier: {:>30}".format(
                    key, results_dict[key][0], results_dict[key][1]))

        # Prints all NOT-Matches next
        print("\n NOT A MATCH:")
        for key in results_dict:

            # Prints only if NOT-a-Match Index 2 == 0
            if results_dict[key][2] == 0:

                # Increments Not-a-Match counter
                n_notmatch += 1
                print("\n{:>30}: \nReal: {:>26}   Classifier: {:>30}".format(
                    key, results_dict[key][0], results_dict[key][1]))

        # Prints Total Number of Images - expects 40 from pet_images folder
        print("\n# Total Images", n_match + n_notmatch, "# Matches:", n_match,
              "# NOT Matches:", n_notmatch)


def check_classifying_labels_as_dogs(results_dict):
    """    For Lab: Classifying Images - 13. Classifying Labels as Dogs
    Prints Pet Image Label, Classifier Label, whether Pet Label is-a-dog(1=Yes,
    0=No), and whether Classifier Label is-a-dog(1=Yes, 0=No) for ALL Matches 
    followed by ALL NOT matches. Next prints out the total number of images 
    followed by how many were matches and how many were not-matches to check 
    all 40 images are processed. Assumes you defined the results_dict dictionary
    as was outlined in '13. Classifying Labels as Dogs'
    Parameters:
      results_dict - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)   where 1 = match between pet image and 
                    classifer labels and 0 = no match between labels
                    idx 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                    idx 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
    Returns:
     Nothing - just prints to console  

    """
    if results_dict is None:
        print(
            "* Doesn't Check the Results Dictionary because 'adjust_results4_isadog' hasn't been defined."
        )
    elif len(results_dict[next(iter(results_dict))]) < 4:
        print(
            "* Doesn't Check the Results Dictionary because 'adjust_results4_isadog' hasn't been defined."
        )

    else:
        # Code for checking adjust_results4_isadog -
        # Checks matches and not matches are classified correctly as "dogs" and
        # "not-dogs" Checks that all 40 images are classified as a Match or Not-a
        # Match

        # Sets counters for matches & NOT-matches
        n_match = 0
        n_notmatch = 0

        # Prints all Matches first
        print("\n     MATCH:")
        for key in results_dict:

            # Prints only if a Match Index 2 == 1
            if results_dict[key][2] == 1:

                # Increments Match counter
                n_match += 1
                print(
                    "\n{:>30}: \nReal: {:>26}   Classifier: {:>30}  \nPetLabelDog: {:1d}  ClassLabelDog: {:1d}"
                    .format(key, results_dict[key][0], results_dict[key][1],
                            results_dict[key][3], results_dict[key][4]))

        # Prints all NOT-Matches next
        print("\n NOT A MATCH:")
        for key in results_dict:

            # Prints only if NOT-a-Match Index 2 == 0
            if results_dict[key][2] == 0:

                # Increments Not-a-Match counter
                n_notmatch += 1
                print(
                    "\n{:>30}: \nReal: {:>26}   Classifier: {:>30}  \nPetLabelDog: {:1d}  ClassLabelDog: {:1d}"
                    .format(key, results_dict[key][0], results_dict[key][1],
                            results_dict[key][3], results_dict[key][4]))

        # Prints Total Number of Images - expects 40 from pet_images folder
        print("\n# Total Images", n_match + n_notmatch, "# Matches:", n_match,
              "# NOT Matches:", n_notmatch)


def check_calculating_results(results_dict, results_stats_dict):

    if results_stats_dict is None:
        print(
            "* Doesn't Check the Results Dictionary because 'calculates_results_stats' hasn't been defined."
        )
    else:
        # Code for checking results_stats_dict -
        # Checks calculations of counts & percentages BY using results_dict
        # to re-calculate the values and then compare to the values
        # in results_stats_dict

        # Initialize counters to zero and number of images total
        n_images = len(results_dict)
        n_pet_dog = 0
        n_class_cdog = 0
        n_class_cnotd = 0
        n_match_breed = 0

        # Interates through results_dict dictionary to recompute the statistics
        # outside of the calculates_results_stats() function
        for key in results_dict:

            # match (if dog then breed match)
            if results_dict[key][2] == 1:

                # isa dog (pet label) & breed match
                if results_dict[key][3] == 1:
                    n_pet_dog += 1

                    # isa dog (classifier label) & breed match
                    if results_dict[key][4] == 1:
                        n_class_cdog += 1
                        n_match_breed += 1

                # NOT dog (pet_label)
                else:

                    # NOT dog (classifier label)
                    if results_dict[key][4] == 0:
                        n_class_cnotd += 1

            # NOT - match (not a breed match if a dog)
            else:

                # NOT - match
                # isa dog (pet label)
                if results_dict[key][3] == 1:
                    n_pet_dog += 1

                    # isa dog (classifier label)
                    if results_dict[key][4] == 1:
                        n_class_cdog += 1

                # NOT dog (pet_label)
                else:

                    # NOT dog (classifier label)
                    if results_dict[key][4] == 0:
                        n_class_cnotd += 1

        # calculates statistics based upon counters from above
        n_pet_notd = n_images - n_pet_dog
        pct_corr_dog = (n_class_cdog / n_pet_dog) * 100
        pct_corr_notdog = (n_class_cnotd / n_pet_notd) * 100
        pct_corr_breed = (n_match_breed / n_pet_dog) * 100

        # prints calculated statistics
        print("\n ** Statistics from calculates_results_stats() function:")
        print(
            "N Images: {:2d}  N Dog Images: {:2d}  N NotDog Images: {:2d} \nPct Corr dog: {:5.1f} Pct Corr NOTdog: {:5.1f}  Pct Corr Breed: {:5.1f}"
            .format(results_stats_dict['n_images'],
                    results_stats_dict['n_dogs_img'],
                    results_stats_dict['n_notdogs_img'],
                    results_stats_dict['pct_correct_dogs'],
                    results_stats_dict['pct_correct_notdogs'],
                    results_stats_dict['pct_correct_breed']))
        print(
            "\n ** Check Statistics - calculated from this function as a check:"
        )
        print(
            "N Images: {:2d}  N Dog Images: {:2d}  N NotDog Images: {:2d} \nPct Corr dog: {:5.1f} Pct Corr NOTdog: {:5.1f}  Pct Corr Breed: {:5.1f}"
            .format(n_images, n_pet_dog, n_pet_notd, pct_corr_dog,
                    pct_corr_notdog, pct_corr_breed))


## Checking if a given input is dog or not


In [86]:
# PROGRAMMER: Abhijith Ganesh
# DATE CREATED: 28 November
# REVISED DATE:
# PURPOSE: Create a function adjust_results4_isadog that adjusts the results
#          dictionary to indicate whether or not the pet image label is of-a-dog,
#          and to indicate whether or not the classifier image label is of-a-dog.
#          All dog labels from both the pet images and the classifier function
#          will be found in the dognames.txt file. We recommend reading all the
#          dog names in dognames.txt into a dictionary where the 'key' is the
#          dog name (from dognames.txt) and the 'value' is one. If a label is
#          found to exist within this dictionary of dog names then the label
#          is of-a-dog, otherwise the label isn't of a dog. Alternatively one
#          could also read all the dog names into a list and then if the label
#          is found to exist within this list - the label is of-a-dog, otherwise
#          the label isn't of a dog.
#         This function inputs:
#            -The results dictionary as results_dict within adjust_results4_isadog
#             function and results for the function call within main.
#            -The text file with dog names as dogfile within adjust_results4_isadog
#             function and in_arg.dogfile for the function call within main.
#           This function uses the extend function to add items to the list
#           that's the 'value' of the results dictionary. You will be adding the
#           whether or not the pet image label is of-a-dog as the item at index
#           3 of the list and whether or not the classifier label is of-a-dog as
#           the item at index 4 of the list. Note we recommend setting the values
#           at indices 3 & 4 to 1 when the label is of-a-dog and to 0 when the
#           label isn't a dog.
#
##
# TODO 4: Define adjust_results4_isadog function below, specifically replace the None
#       below by the function definition of the adjust_results4_isadog function.
#       Notice that this function doesn't return anything because the
#       results_dict dictionary that is passed into the function is a mutable
#       data type so no return is needed.
#
def adjust_results4_isadog(results_dict, dogfile):
    print(dogfile)
    with open(dogfile) as fileObj:
        data = fileObj.readlines()
        _data = []
        for i in data:
            _data.extend(i.strip("\n").strip(" ").split(","))

    for i in results_dict.values():
        key = i[0]
        key_2 = i[1]

        if key in _data:
            if len(i) < 4:
                i.insert(3, 1)

        else:
            if len(i) < 4:
                i.insert(3, 0)

    # Insert operation because direct access is forbidden
        if key_2 in _data:
            if len(i) < 4:
                i.insert(3, 1)

        else:
            if len(i) < 4:
                i.insert(3, 0)
    """
    Adjusts the results dictionary to determine if classifier correctly 
    classified images 'as a dog' or 'not a dog' especially when not a match. 
    Demonstrates if model architecture correctly classifies dog images even if
    it gets dog breed wrong (not a match).
    Parameters:
      results_dict - Dictionary with 'key' as image filename and 'value' as a 
                    List. Where the list will contain the following items: 
                  index 0 = pet image label (string)
                  index 1 = classifier label (string)
                  index 2 = 1/0 (int)  where 1 = match between pet image
                    and classifer labels and 0 = no match between labels
                ------ where index 3 & index 4 are added by this function -----
                 NEW - index 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                 NEW - index 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
     dogfile - A text file that contains names of all dogs from the classifier
               function and dog names from the pet image files. This file has 
               one dog name per line dog names are all in lowercase with 
               spaces separating the distinct words of the dog name. Dog names
               from the classifier function can be a string of dog names separated
               by commas when a particular breed of dog has multiple dog names 
               associated with that breed (ex. maltese dog, maltese terrier, 
               maltese) (string - indicates text file's filename)
    Returns:
           None - results_dict is mutable data type so no return needed.
    """
    None


adjust_results4_isadog(results_dict, "dognames.txt")


dognames.txt


In [22]:
# TODO 5: Caclualte Stats


def calculates_results_stats(results_dict):
    # Initializing empty dict
    results_stats_dict: dict = {}

    for key in results_dict.keys():
        results_stats_dict[key] = 0

    for key, val in results_dict.items():
        results_stats_dict[key] = abs(val[4] - val[3])
    '''
    Result Stats will hold the following data as mentioned in Markdown
    '''
    # Replace None with the results_stats_dict dictionary that you created with
    # this function
    return results_stats_dict


results_stats_dict = calculates_results_stats(results_dict)


### Representation of Stats Dict

```json
{
    key:value
}
key => fileName
value => MisMatch between prediction and reality
```


In [82]:
# */AIPND-revision/intropyproject-classify-pet-images/print_results.py
#
# PROGRAMMER: Abhijith Ganesh
# DATE CREATED: 28 November
# REVISED DATE:
# PURPOSE: Create a function print_results that prints the results statistics
#          from the results statistics dictionary (results_stats_dict). It
#          should also allow the user to be able to print out cases of misclassified
#          dogs and cases of misclassified breeds of dog using the Results
#          dictionary (results_dict).
#         This function inputs:
#            -The results dictionary as results_dict within print_results
#             function and results for the function call within main.
#            -The results statistics dictionary as results_stats_dict within
#             print_results function and results_stats for the function call within main.
#            -The CNN model architecture as model wihtin print_results function
#             and in_arg.arch for the function call within main.
#            -Prints Incorrectly Classified Dogs as print_incorrect_dogs within
#             print_results function and set as either boolean value True or
#             False in the function call within main (defaults to False)
#            -Prints Incorrectly Classified Breeds as print_incorrect_breed within
#             print_results function and set as either boolean value True or
#             False in the function call within main (defaults to False)
#         This function does not output anything other than printing a summary
#         of the final results.
##
#  TODO 6: Define print_results function below, specifically replace the None
#       below by the function definition of the print_results function.
#       Notice that this function doesn't to return anything because it
#       prints a summary of the results using results_dict and results_stats_dict
#
def print_results(results_dict,
                  results_stats_dict,
                  model,
                  print_incorrect_dogs=False,
                  print_incorrect_breed=False):
    """
    Prints summary results on the classification and then prints incorrectly 
    classified dogs and incorrectly classified dog breeds if user indicates 
    they want those printouts (use non-default values)
    Parameters:
      results_dict - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)  where 1 = match between pet image and 
                            classifer labels and 0 = no match between labels
                    idx 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                    idx 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
      results_stats_dict - Dictionary that contains the results statistics (either
                   a  percentage or a count) where the key is the statistic's 
                     name (starting with 'pct' for percentage or 'n' for count)
                     and the value is the statistic's value 
      model - Indicates which CNN model architecture will be used by the 
              classifier function to classify the pet images,
              values must be either: resnet alexnet vgg (string)
      print_incorrect_dogs - True prints incorrectly classified dog images and 
                             False doesn't print anything(default) (bool)  
      print_incorrect_breed - True prints incorrectly classified dog breeds and 
                              False doesn't print anything(default) (bool) 
    Returns:
           None - simply printing results.
    """

    dogs: int = 0
    mismatch:int = 0

    for i in results_dict.values():
        if (i[4] != 0):
            dogs += i[4]

    print(f"Total Images: {len(results_dict)}")
    print(f"Dog Images: {dogs}")
    print(f"Not-a-Dog Images: {len(results_dict)-dogs}")
    
    for i in results_stats_dict.values():
        mismatch += i

    print(mismatch, len(results_stats_dict))


print_results(results_dict, results_stats_dict, None, None)


Total Images: 40
Dog Images: 10
Not-a-Dog Images: 30
5 40


In [238]:
# PROGRAMMER: Abhijith Ganesh
# DATE CREATED: 28 November
# REVISED DATE:
# PURPOSE: Classifies pet images using a pretrained CNN model, compares these
#          classifications to the true identity of the pets in the images, and
#          summarizes how well the CNN performed on the image classification task.
#          Note that the true identity of the pet (or object) in the image is
#          indicated by the filename of the image. Therefore, your program must
#          first extract the pet image label from the filename before
#          classifying the images using the pretrained CNN model. With this
#          program we will be comparing the performance of 3 different CNN model
#          architectures to determine which provides the 'best' classification.
#
# Use argparse Expected Call with <> indicating expected user input:
#      python check_images.py --dir <directory with images> --arch <model>
#             --dogfile <file that contains dognames>
#   Example call:
#    python check_images.py --dir pet_images/ --arch vgg --dogfile dognames.txt
##

# Imports python modules
from time import time, sleep


def main():
    # TODO 0: Measures total program runtime by collecting start time
    start_time = time()

    # TODO 1: Define get_input_args function within the file get_input_args.py
    # This function retrieves 3 Command Line Arugments from user as input from
    # the user running the program from a terminal window. This function returns
    # the collection of these command line arguments from the function call as
    # the variable in_arg
    in_arg = get_input_args()

    # Function that checks command line arguments using in_arg
    check_command_line_arguments(in_arg)

    # TODO 2: Define get_pet_labels function within the file get_pet_labels.py
    # Once the get_pet_labels function has been defined replace 'None'
    # in the function call with in_arg.dir  Once you have done the replacements
    # your function call should look like this:
    #             get_pet_labels(in_arg.dir)
    # This function creates the results dictionary that contains the results,
    # this dictionary is returned from the function call as the variable results
    results = get_pet_labels(in_arg.dir)

    # Function that checks Pet Images in the results Dictionary using results
    check_creating_pet_image_labels(results)

    # TODO 3: Define classify_images function within the file classiy_images.py
    # Once the classify_images function has been defined replace first 'None'
    # in the function call with in_arg.dir and replace the last 'None' in the
    # function call with in_arg.arch  Once you have done the replacements your
    # function call should look like this:
    #             classify_images(in_arg.dir, results, in_arg.arch)
    # Creates Classifier Labels with classifier function, Compares Labels,
    # and adds these results to the results dictionary - results
    classify_images(in_arg.dir, results, in_arg.arch)

    # Function that checks Results Dictionary using results
    check_classifying_images(results)

    # TODO 4: Define adjust_results4_isadog function within the file adjust_results4_isadog.py
    # Once the adjust_results4_isadog function has been defined replace 'None'
    # in the function call with in_arg.dogfile  Once you have done the
    # replacements your function call should look like this:
    #          adjust_results4_isadog(results, in_arg.dogfile)
    # Adjusts the results dictionary to determine if classifier correctly
    # classified images as 'a dog' or 'not a dog'. This demonstrates if
    # model can correctly classify dog images as dogs (regardless of breed)
    adjust_results4_isadog(results, in_arg.dogfile)

    # Function that checks Results Dictionary for is-a-dog adjustment using results
    check_classifying_labels_as_dogs(results)

    # TODO 5: Define calculates_results_stats function within the file calculates_results_stats.py
    # This function creates the results statistics dictionary that contains a
    # summary of the results statistics (this includes counts & percentages). This
    # dictionary is returned from the function call as the variable results_stats
    # Calculates results of run and puts statistics in the Results Statistics
    # Dictionary - called results_stats
    results_stats = calculates_results_stats(results)

    # Function that checks Results Statistics Dictionary using results_stats
    check_calculating_results(results, results_stats)

    # TODO 6: Define print_results function within the file print_results.py
    # Once the print_results function has been defined replace 'None'
    # in the function call with in_arg.arch  Once you have done the
    # replacements your function call should look like this:
    #      print_results(results, results_stats, in_arg.arch, True, True)
    # Prints summary results, incorrect classifications of dogs (if requested)
    # and incorrectly classified breeds (if requested)
    print_results(results, results_stats, in_arg.arch, True, True)

    end_time = time()

    tot_time = end_time - start_time

    print(
        "\n** Total Elapsed Runtime:",
        str(int((tot_time / 3600))) + ":" + str(int(
            (tot_time % 3600) / 60)) + ":" + str(int((tot_time % 3600) % 60)))


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--dir DIR] [--arch ARCH]
                             [--dogfile DOGFILE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"d7b7ebe9-9731-4f84-a6dc-f507f2d3d092" --shell=9007 --transport="tcp" --iopub=9009 --f="c:\Users\Abhijith G\AppData\Roaming\jupyter\runtime\kernel-v2-30884NFwA6IoOhoq8.json"


SystemExit: 2

c:\Users\Abhijith G\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
